In [ ]:
import numpy as np

In [ ]:
def patch_accuracy(mask_method, oracle_T_dimension):
    """
    Computes the patch accuracy

    Parameters:
    mask_method (numpy.ndarray): The method mask of shape (T, H, W).
    mask_oracle (numpy.ndarray): The oracle mask of shape (T).

    Returns:
    float: The ratio of relevant masks in total masks
    """
    assert mask_method.shape[0] == oracle_T_dimension.shape[0], "Masks must have the same shape"
    return np.sum(mask_method[oracle_T_dimension, :, :]) / np.sum(mask_method)
    